<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Jan Attig</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; font-size: 30px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; font-size: 30px; padding:0px;">Übungsblatt 9</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; font-size: 20px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>
<hr>

**Website** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Abgabe**:  22.6.2020 bis 13:00 Uhr

**Besprechung**:  24.6.2020 - 26.6.2020

**Name**: <font color="red">Bitte geben Sie ihren Namen an</font>

**Matrikelnummer**: <font color="red">Bitte geben Sie ihre Matrikelnummer an</font>

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 25px; font-weight: bold; text-decoration: underline">Aufgaben auf Übungsblatt 9</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px"></td></tr></table>

- Hofstadter Butterfly [8 Punkte]
- **Der Geist in der Matrix** [12 Punkte]
- (Bonus) Wie eine Düne im Wind [10 Bonus-Punkte]


<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; text-align: left; font-size: 25px; font-weight: bold; text-decoration: underline">Der Geist in der Matrix</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 15px">[12 Punkte]</td></tr></table>

In der nachfolgenden Aufgabe wollen wir uns mit der numerischen Berechnung von Eigenwerten mittels Lanczos Algorithmus beschäftigen. Dafür werden Sie zuerst eine einfache Matrix mittels **Power-Methode** untersuchen und schließlich in einem zweiten Teil einen gegebenen Hamiltonian mittels **Lanczos Algorithmus**.

### Teil A: Power-Methode
Der erste Schritt auf dem Weg zum Verständnis des Lanczos Algorithmus ist die **Power-Methode**. Wir beginnen mit einer Matrix $H$ und einem Startvektor $b_0$. Diesen Vektor iterieren wir mithilfe der Relation

\begin{equation}
b_{k + 1} = \dfrac{H b_k}{|H b_k|}
\end{equation}

In dieser Aufgabe starten wir mit dem Vektor $\left(1, 1, 1, 1\right)^T$ und suchen nach dem größten Eigenwert der Matrix
\begin{equation}
H = 
\begin{pmatrix}
1 && 1 && 1 && 1 \\
1.1 && 1.2 && 1.3 && 1.4 \\
1.1 && 1.4 && 1.9  && 1.16 \\
1.1 && 1.8 && 1.27 && 1.64
\end{pmatrix}
\end{equation}

(1) Berechnen Sie die Eigenwerte und Eigenvektoren der Matrix mittels der Julia Funktion *eig*

(2) Iterieren Sie den Startvektor gemäß der oben angegebenen Vorschrift und plotten Sie die Norm des Differenzvektors zwischen $b_k$ und dem korrespondierenden exakten Eigenvektor als Funktion des Iterationsschritts $k$.

### Teil B: Lanczos

Wie Sie in der Vorlesung gesehen haben, funktioniert der **Lanczos**-Algorithmus sehr ähnlich, erlaubt aber die Bestimmung mehrerer Eigenwerte und Eigenvektoren. Insbesondere in der Vielteilchenphysik ist das Lanczos-Verfahren ein absolutes Standardverfahren, um exakte Informationen über ein Modell zu erhalten. 

In dieser Aufgabe betrachten wir eine Kette aus sogenannten *hard core bosons*, die wir mit einem Hamiltonian beschreiben, der Ihnen vielleicht vom harmonischen Oszillator ein wenig vertraut erscheinen mag:

\begin{equation}
H = -t\sum\limits_{\langle i, j \rangle}\left( a_i^\dagger a_j^{\phantom\dagger} + h.c.\right) + V\sum\limits_{\langle i, j \rangle}n_i n_j
\end{equation}

Die Summen laufen über benachbarte Gitterplätze $i$ und $j$, was typischerweise durch die Klammer $\langle \dots\rangle$ angezeigt wird. In der ersten Summe werden nun Teilchen auf Platz $j$ durch den Operator $a_j^{\phantom\dagger}$ vernichtet und am Platz $i$ mit $a_i^\dagger$ erzeugt. Er beschreibt also einen Hüpfvorgang und steht für die kinetische Energie des Systems. Mit dem negativen Vorfaktor $-t$ erkennen wir, dass die Energie niedriger ist, je mobiler die Teilchen sind. In der zweiten Summe werden nun die Teilchenzahlen $n_i$ auf benachbarten Gitterplätzen betrachtet, weshalb dieser Teil die potentielle Energie des Systems darstellt. 

Eine Besonderheit, die das Aufstellen des Hamiltonians erleichtert ist, dass hard core bosonen zwar bosonischen Kommutationsregeln unterliegen, es aber dennoch jeder Zustand nur durch maximal ein Boson besetzt werden darf, sie also einer Art Pauli-Prinzip unterliegen. Das Austellen der Matrix geschieht in dem untenstehenden, ausführlich kommentierten Code.

In [ ]:
# Anzahl der Plaetze auf der Kette
N = 8 
# Anzahl der Teilchen
p = Int(N / 2) 

# Groesse des Hilbertraum, verteile p Teilchen auf N Plaetze
n_states = binomial(N, p)
# Array der die Zustaende speichert
states = zeros(Int8, N, n_states)
# Zaehler fuer Zustand
idx = [0]

t = -1. # Hoppingstaerke ~ kinetische Energie
V = -1. # Wechselwirkung ~ potentielle Energie

# Funktion wird rekursiv aufgerufen
function fill(state, pos, N, p, idx, states)
    # Abbruch wenn Gitterplatz ausserhalb der Kette gefuellt werden soll
    if pos > N
        # stimmt die Teilchenzahl?
        if sum(state) == p
            #speichere Zustand und erhoehe Zaehler
            idx[1] += 1
            states[:, idx[1]] = state
        end
        return
    end
        
    # lasse Position pos + 1 frei
    fill(copy(state), pos + 1, N, p, idx, states)
    # besetze Position pos + 1
    state[pos] = 1
    fill(copy(state), pos + 1, N, p, idx, states)
end

# erzeuge einen Referenzzustand
state = zeros(Int, N)
# fuelle Zustandsmatrix
fill(copy(state), 1, N, p, idx, states)

# der Hamiltonian ist eine duenn besetzte = sparse Matrix
H = zeros(n_states, n_states)

# iteriere ueber alle Zustandskombinationen
for k in 1:n_states
    for b in 1:n_states
        
        # Wende Hamiltonian an
        for i in 1:N - 1
            ket = copy(states[:, k])
            bra = copy(states[:, b])
            
            # huepfen nur, wenn Zielplatz frei und Ausgangsplatz frei
            if (ket[i] == 0 && ket[i + 1] == 1) || (ket[i] == 1 && ket[i + 1] == 0)
                ket[i] = mod(ket[i] + 1, 2)
                ket[i+1] = mod(ket[i] + 1, 2)
                
                # Sind bra und ket gleich? ^= Skalarprodukt
                if maximum(abs.(ket - bra)) == 0
                    H[b, k] += t
                    H[k, b] += t
                end
            end
            
            # Wechselwirkung, wenn beide Plaetze besetzt
            if ket[i] == 1 && ket[i + 1] == 1
                # Sind bra und ket gleich? ^= Skalarprodukt
                if maximum(abs.(ket - bra)) == 0
                    H[b, k] += V
                    H[k, b] += V
                end
            end
        end
    end
end

Diagonlisieren Sie mithilfe des Lanczos-Algorithmus diese Matrix iterativ mit 300 Iterationsschritten. Speichern Sie während des Iterationsvorgangs die ersten 20 Eigenwerte ab. Beachten Sie, dass der j-te Eigenwert erst ab dem j-ten Iterationsschritt zur Verfügung steht. Plotten Sie dann den Verlauf dieser Eigenwerte als Funktion des Iterationsschritts. Beschreiben Sie das Konvergenzverhalten. Konvergieren alle gleichzeitig? Ist das Ergebnis für alle Eigenwerte stabil?

**Hinweis**

Um die Berechnung der Eigenwerte zu beschleunigen, können Sie in Ihrem Algorithmus den Typ *SymTridiagonal* verwenden, der dann von Julia optimiert verarbeitet wird. Als Paramter übergeben Sie die Diagonale, sowie eine der Nebendiagonalen: